#### Ensemble - ExtraTree
 - 배깅 방식의 앙상블 ==> 중복 허용한 랜덤 샐픔 + 동일 모델(DecisionTree)
   * 대표 알고리즘 : RandomForestC/R
 - 페이스트 방식의 앙상블 ==> 랜덤 샘플 + 동일 모델(DecisionTree)
   * 대표 알고리즘 : ExtraTreeC/R

- 와인분류 => 0과 1 2개 종류 분류

[1] 모듈 로딩 및 데이터 준비

In [33]:
#  모듈로딩
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
# 데이터
wineDF = pd.read_csv(r'C:\Users\KDP-17\EX_PANDAS6\MachineLearning\data\wine.csv')
wineDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [35]:
# 타겟/라벨의 클래스 분포
wineDF['class'].value_counts()

class
1.0    4898
0.0    1599
Name: count, dtype: int64

In [36]:
wineDF.describe()

featureDF = wineDF[wineDF.columns[:-1]]
targetSR =  wineDF[wineDF.columns[-1]]

print(f'featureDF : {featureDF.shape}, targetSr : {targetSR.shape}')



featureDF : (6497, 3), targetSr : (6497,)


[2] 학습 준비

In [37]:
# 학습용 & 테스트용 데이터셋 분할
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetSR,
                                                    test_size=0.2,
                                                    stratify=targetSR,
                                                    random_state=10)

In [39]:
print(f'X_train : {X_train.shape}, y_train : {y_train.shape}')
print(f'X_test : {X_test.shape}, y_test : {y_test.shape}')

X_train : (5197, 3), y_train : (5197,)
X_test : (1300, 3), y_test : (1300,)


[3] 학습 진행

In [40]:
# 학습 방법 : 지도학습 > 분류
# 알고리즘 : 앙상블 > 배깅 - RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [41]:
# 인스턴스 객체 생성 => 100개의 내부 DT 모델에서 사용할 데이터셋 생성
#                    random state 매개변수 설정으로 고정된 데이터셋 생성
#                    oob_score 매개변수 : 샘플 데이터셋 추출 후 남은 데이터셋 검증용으로 사용
rf_model = ExtraTreesClassifier(random_state=7)

# 학습
rf_model.fit(X_train,y_train)

ExtraTreesClassifier(random_state=7)

In [42]:
# 모델 파라미터
print(f'classes_ : {rf_model.classes_}')
print(f'n_classes_ : {rf_model.n_classes_}개')
print(f'feature_names_in_ : {rf_model.feature_names_in_}')
print(f'n_features_in_ : {rf_model.n_features_in_}개')
print(f'feature_importances_ : {rf_model.feature_importances_}')

classes_ : [0. 1.]
n_classes_ : 2개
feature_names_in_ : ['alcohol' 'sugar' 'pH']
n_features_in_ : 3개
feature_importances_ : [0.18859051 0.52297152 0.28843797]


In [43]:
# 모델 파라미터 
print(f'       : {rf_model.estimator_}')
for est in rf_model.estimators_: print(est)


       : ExtraTreeClassifier()
ExtraTreeClassifier(random_state=327741615)
ExtraTreeClassifier(random_state=976413892)
ExtraTreeClassifier(random_state=1202242073)
ExtraTreeClassifier(random_state=1369975286)
ExtraTreeClassifier(random_state=1882953283)
ExtraTreeClassifier(random_state=2053951699)
ExtraTreeClassifier(random_state=959775639)
ExtraTreeClassifier(random_state=1956722279)
ExtraTreeClassifier(random_state=2052949340)
ExtraTreeClassifier(random_state=1322904761)
ExtraTreeClassifier(random_state=165338510)
ExtraTreeClassifier(random_state=1133316631)
ExtraTreeClassifier(random_state=4812360)
ExtraTreeClassifier(random_state=372560217)
ExtraTreeClassifier(random_state=309457262)
ExtraTreeClassifier(random_state=1801189930)
ExtraTreeClassifier(random_state=1152936666)
ExtraTreeClassifier(random_state=68334472)
ExtraTreeClassifier(random_state=2146978983)
ExtraTreeClassifier(random_state=119248870)
ExtraTreeClassifier(random_state=769786948)
ExtraTreeClassifier(random_state=1583

[4] 성능 평가

In [44]:
train_score = rf_model.score(X_train,y_train)
test_score = rf_model.score(X_test,y_test)

In [45]:
print(f'train_score : {train_score}, test_score : {test_score}')

train_score : 0.9976909755628247, test_score : 0.8853846153846154


[5] 튜닝
 - RandomizedSearchCV 하이퍼파라미터 최적화 클래스
   * 범위가 넓은 하이퍼파라미터 설정에 좋음
   * 지정된 범윙에서 지정된 횟수만큼 하이퍼파라미터를 추출하여 조합 진행

In [46]:
# 모듈로딩
from sklearn.model_selection import RandomizedSearchCV

In [47]:
# RandomForestClassifier 하이퍼파라미터 설정
params = {'max_depth':range(2,16),
          'min_samples_leaf':range(5,16)}
        

In [48]:
rf_model = RandomForestClassifier(random_state=7)

In [49]:
searchCV = RandomizedSearchCV(rf_model,param_distributions=params,
                              n_iter=50,
                              verbose=4)

In [50]:
searchCV.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END ...max_depth=9, min_samples_leaf=5;, score=0.875 total time=   0.2s
[CV 2/5] END ...max_depth=9, min_samples_leaf=5;, score=0.873 total time=   0.2s
[CV 3/5] END ...max_depth=9, min_samples_leaf=5;, score=0.880 total time=   0.2s
[CV 4/5] END ...max_depth=9, min_samples_leaf=5;, score=0.864 total time=   0.2s
[CV 5/5] END ...max_depth=9, min_samples_leaf=5;, score=0.877 total time=   0.2s
[CV 1/5] END ..max_depth=5, min_samples_leaf=13;, score=0.869 total time=   0.1s
[CV 2/5] END ..max_depth=5, min_samples_leaf=13;, score=0.853 total time=   0.1s
[CV 3/5] END ..max_depth=5, min_samples_leaf=13;, score=0.862 total time=   0.1s
[CV 4/5] END ..max_depth=5, min_samples_leaf=13;, score=0.856 total time=   0.1s
[CV 5/5] END ..max_depth=5, min_samples_leaf=13;, score=0.857 total time=   0.1s
[CV 1/5] END .max_depth=10, min_samples_leaf=14;, score=0.875 total time=   0.2s
[CV 2/5] END .max_depth=10, min_samples_leaf=14

RandomizedSearchCV(estimator=RandomForestClassifier(random_state=7), n_iter=50,
                   param_distributions={'max_depth': range(2, 16),
                                        'min_samples_leaf': range(5, 16)},
                   verbose=4)

In [51]:
# 모델 파라미터
print(f'searchCV.best_score : {searchCV.best_score_}')
print(f'searchCV.best_params_ : {searchCV.best_params_}')
print(f'searchCV.best_estimator_ : {searchCV.best_estimator_}')

cv_resulDF = pd.DataFrame(searchCV.cv_results_)

cv_resulDF

searchCV.best_score : 0.8778140963944621
searchCV.best_params_ : {'min_samples_leaf': 5, 'max_depth': 11}
searchCV.best_estimator_ : RandomForestClassifier(max_depth=11, min_samples_leaf=5, random_state=7)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.286560,0.014923,0.016693,0.002657,5,9,"{'min_samples_leaf': 5, 'max_depth': 9}",0.875000,0.873077,0.879692,0.864293,0.876805,0.873773,0.005217,3
1,0.205796,0.008799,0.011900,0.006016,13,5,"{'min_samples_leaf': 13, 'max_depth': 5}",0.869231,0.852885,0.862368,0.855630,0.856593,0.859341,0.005831,41
2,0.249283,0.004730,0.021149,0.006338,14,10,"{'min_samples_leaf': 14, 'max_depth': 10}",0.875000,0.866346,0.864293,0.858518,0.871030,0.867037,0.005656,26
3,0.257666,0.003983,0.020795,0.003703,11,14,"{'min_samples_leaf': 11, 'max_depth': 14}",0.873077,0.864423,0.872955,0.860443,0.873917,0.868963,0.005488,11
4,0.231550,0.009888,0.016481,0.002236,14,7,"{'min_samples_leaf': 14, 'max_depth': 7}",0.874038,0.859615,0.863330,0.856593,0.863330,0.863381,0.005897,34
5,0.243964,0.006837,0.015327,0.001706,9,7,"{'min_samples_leaf': 9, 'max_depth': 7}",0.874038,0.864423,0.865255,0.855630,0.871030,0.866075,0.006332,31
6,0.259139,0.006043,0.007049,0.005017,15,11,"{'min_samples_leaf': 15, 'max_depth': 11}",0.871154,0.863462,0.865255,0.862368,0.875842,0.867616,0.005110,19
7,0.263931,0.001604,0.009387,0.006472,15,13,"{'min_samples_leaf': 15, 'max_depth': 13}",0.871154,0.860577,0.865255,0.866218,0.871030,0.866847,0.003957,27
8,0.226288,0.010113,0.018105,0.002039,9,6,"{'min_samples_leaf': 9, 'max_depth': 6}",0.869231,0.861538,0.863330,0.857555,0.864293,0.863189,0.003800,35
9,0.284295,0.004712,0.017740,0.001771,8,11,"{'min_samples_leaf': 8, 'max_depth': 11}",0.877885,0.870192,0.872955,0.869105,0.878730,0.873773,0.003918,3
